In [1]:
from FusionNet import * 
#from UNet import *
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# hyperparameters
batch_size = 1
img_size = 256
#img_size = 1024ㅣ
lr = 0.0002
epoch = 1


In [3]:
import shutil, os

train_img_dir = "train/"
dir = os.path.join(train_img_dir + '.ipynb_checkpoints')
if os.path.exists(dir) and os.path.isdir(dir):
        shutil.rmtree(dir)
train_img_data = dset.ImageFolder(root=train_img_dir, transform = transforms.Compose([
                                            transforms.Resize(size=img_size),
                                            transforms.CenterCrop(size=(img_size,img_size*2)),
                                            transforms.ToTensor(),
                                            ]))
train_img_batch = data.DataLoader(train_img_data, batch_size=batch_size,
                            shuffle=True, num_workers=2)

val_img_dir = "val/"
dir = os.path.join(val_img_dir + '.ipynb_checkpoints')
if os.path.exists(dir) and os.path.isdir(dir):
        shutil.rmtree(dir)
val_img_data = dset.ImageFolder(root=val_img_dir, transform = transforms.Compose([
                                            transforms.Resize(size=img_size),
                                            transforms.CenterCrop(size=(img_size,img_size)),
                                            transforms.ToTensor(),
                                            ]))
val_img_batch = data.DataLoader(val_img_data, batch_size=batch_size,
                            shuffle=False, num_workers=2)



In [4]:
#generator = UnetGenerator(3,3,64)

generator = nn.DataParallel(FusionGenerator(3,3,64),device_ids=[i for i in range(1)]).cuda()


------Initiating FusionNet------



RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
#torch.nn.Module.dump_patches = True
try:
    generator = torch.load('./model/{}.pkl'.format('nllfusionnet'))
    print("\n--------model restored--------\n")
except:
    print("\n--------model not restored--------\n")
    pass


--------model not restored--------



In [ ]:
# loss function & optimizer
recon_loss_func = nn.MSELoss()
gen_optimizer = torch.optim.Adam(generator.parameters(),lr=lr)


In [ ]:
# training
import time
try:
    file = open('./{}_mse_fu_loss'.format('fusionnet'), 'w')
except:
    time.pause(60)
    file = open('./{}_mse_fu_loss'.format('fusionnet'), 'w')
               
    

for i in range(epoch):
    for _,(image,label) in enumerate(train_img_batch):
        satel_image, map_image = torch.chunk(image, chunks=2, dim=3) 
        
        gen_optimizer.zero_grad()

        x = Variable(satel_image).cuda(0)
        y_ = Variable(map_image).cuda(0)
        y = generator.forward(x)
        
        loss = recon_loss_func(y,y_)
        file.write(str(loss)+"\n")
        loss.backward()
        gen_optimizer.step()
      #  torch.save(generator,'./model/nll{}.pkl'.format('fusionnet'))                
        
        if _ % 50 == 0:
            print(i)
            print(loss)
         #   v_utils.save_image(x.cpu().data,"./result_/original_image_{}_{}.png".format(i,_))
         #   v_utils.save_image(y_.cpu().data,"./result_/label_image_{}_{}.png".format(i,_))
         #   v_utils.save_image(y.cpu().data,"./result_/gen_image_{}_{}.png".format(i,_))
            torch.save(generator,'./model/nll{}.pkl'.format('fusionnet'))    
        if _ % 500 == 0:
            print(i)
            print(loss)
         #   v_utils.save_image(x.cpu().data,"./result_/original_image_{}_{}.png".format(i,_))
            v_utils.save_image(y_.cpu().data,"./result_train/label_image_{}_{}.png".format(i,_))
            v_utils.save_image(y.cpu().data,"./result_train/gen_image_{}_{}.png".format(i,_))
         #   torch.save(generator,'./model/nll{}.pkl'.format('fusionnet'))    
        


0
tensor(0.1314, device='cuda:0', grad_fn=<MseLossBackward>)
0
tensor(0.1314, device='cuda:0', grad_fn=<MseLossBackward>)


KeyboardInterrupt: 

In [ ]:
file.write(str(loss)+"\n")

59

In [ ]:
# validate
generator = torch.load('./model/{}.pkl'.format('nllfusionnet'))

#,map_location=lambda storage, loc: storage.cuda())


for _,(image,label) in enumerate(val_img_batch):
    #satel_image, map_image = torch.chunk(image, chunks=2, dim=3) 
    print(_)
    x = Variable(image)
    #y_ = Variable(map_image)
    y = generator.forward(x)

    #v_utils.save_image(x.cpu().data,"./result/original_image_{}.png".format(_))
    #v_utils.save_image(y_.cpu().data,"./result/label_image_{}.png".format(_))
    v_utils.save_image(y.cpu().data,"./result_val/gen_image_{}.png".format(_))

0


In [ ]:
val_img_data

Dataset ImageFolder
    Number of datapoints: 1
    Root location: val/
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=bilinear)
               CenterCrop(size=(256, 256))
               ToTensor()
           )